In [2]:
!pip install rdflib

     |████████████████████████████████| 235kB 12.4MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 


In [48]:
from rdflib import Graph, Namespace, URIRef, Literal
import rdflib

# define the namespaces. Now we can refer to entities 
# in the namespacing using these handles.
BUILDING = Namespace("https://w3id.org/rec/building/#")
CORE = Namespace("https://w3id.org/rec/core/#")
DEVICE = Namespace("https://w3id.org/rec/device/#")
RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
MYBUILDING = Namespace("http://mysite.com/buildings/mybuilding#")

# create the graph and bind the namespace prefixes to it
g = Graph()
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('building', BUILDING)
g.bind('core', CORE)
g.bind('device', DEVICE)
g.bind('mybuilding', MYBUILDING)

# add() inserts another triple into the graph as (subject, predicate, object)
g.add((MYBUILDING.floor_1, RDF.type, BUILDING.Floor))
g.add((MYBUILDING.room_1, RDF.type, BUILDING.Laboratory))
g.add((MYBUILDING.room_2, RDF.type, BUILDING.CookingRoom))
# These would be inferred via a reasoner
g.add((MYBUILDING.room_1, RDF.type, CORE.Room))
g.add((MYBUILDING.room_2, RDF.type, CORE.Room))
g.add((MYBUILDING.temp_sensor1, RDF.type, DEVICE.Sensor))
g.add((MYBUILDING.temp_sensor1, CORE.hasLittera, Literal('LB04_QT_01')))
g.add((MYBUILDING.device1, RDF.type, DEVICE.Device))

EX = MYBUILDING

# add the relationships constituting the other edges in the graph
g.add((EX.room_1, CORE.hasSuperBuildingComponent, EX.floor_1))
g.add((EX.room_2, CORE.hasSuperBuildingComponent, EX.floor_1))
g.add((EX.device1, CORE.isMountedIn, EX.room_2))
g.add((EX.temp_sensor1, DEVICE.hasSuperDevice, EX.device1))


# save the output to "mybuilding.ttl"
g.serialize(destination='mybuilding.ttl',format='turtle')

In [49]:
f = open("mybuilding.ttl", "r")
print(f.read())

@prefix building: <https://w3id.org/rec/building/#> .
@prefix core: <https://w3id.org/rec/core/#> .
@prefix device: <https://w3id.org/rec/device/#> .
@prefix mybuilding: <http://mysite.com/buildings/mybuilding#> .

mybuilding:room_1 a building:Laboratory,
        core:Room ;
    core:hasSuperBuildingComponent mybuilding:floor_1 .

mybuilding:temp_sensor1 a device:Sensor ;
    core:hasLittera "LB04_QT_01" ;
    device:hasSuperDevice mybuilding:device1 .

mybuilding:device1 a device:Device ;
    core:isMountedIn mybuilding:room_2 .

mybuilding:room_2 a building:CookingRoom,
        core:Room ;
    core:hasSuperBuildingComponent mybuilding:floor_1 .

mybuilding:floor_1 a building:Floor .




In [38]:
# Find the floor with SPARQL
qres = g.query("SELECT ?s WHERE {?s a building:Floor}")
for row in qres:
    print("%s" % row)

http://mysite.com/buildings/mybuilding#floor_1


In [39]:
# Find all rooms that are part of floor 1 with SPARQL
qres = g.query("SELECT ?s WHERE {?s core:hasSuperBuildingComponent mybuilding:floor_1}")
for row in qres:
    print("%s" % row)

http://mysite.com/buildings/mybuilding#room_2
http://mysite.com/buildings/mybuilding#room_1


In [47]:
# List all devices and their locations:
qres = g.query("SELECT ?s ?r WHERE {?s core:isMountedIn ?r}")
for row in qres:
    print("Device: %s  in Room: %s" % row)


Device: http://mysite.com/buildings/mybuilding#device1  in Room: http://mysite.com/buildings/mybuilding#room_2
